In [3]:
import pandas as pd

import itertools
import random

from joblib import Parallel, delayed, dump
import multiprocessing
import logging
import sys
import warnings
import time

from sklearn.pipeline import Pipeline
from sklearn.metrics import get_scorer
from xgboost import XGBRegressor
import numpy as np

**La siguiente celda sirve para ver la cantidad de núcleos en la CPU**, para que sepan en cuantos trabajos dividir las tareas. La idea es realizar tantos procesos paralelos como núcleos tenga nuestra CPU (el dataset es chico, asique dudo que haya problemas con la RAM, pero si nos quedamos sin RAM, hay que paralelizar en menos procesos que la cantidad de núcleos, o no hacerlo en absoluto). Esto es para que podamos hacer búsquedas más exhaustivas en menos tiempo. Entiendo que scikit-learn no aprovecha GPU (según chat GPT), pero XGBoost si 👀 (@fede, pero si lo querés aprovechar tendrías que cambiar la función, e indicarlo en un argumento al instanciar la clase$^1$).

$^1$ no entiendo mucho de GPUs, capaz vos entendés más pero comparto lo que leí por si sirve.

In [4]:
print(f"Available CPU cores: {multiprocessing.cpu_count()}")

Available CPU cores: 4


## Función para entrenar y evaluar

In [5]:
def optimizador_hiperparametros(nombre, config, X_train, y_train, X_val, y_val, nombre_dataset, metrica,
                                busqueda='random', random_state=42, n_iter=20, n_jobs=1):
    """
    Función para optimizar los hiperparámetros de un modelo dado sobre el conjunto
    entrenamiento. La evaluación para decidir el mejor modelo se realiza sobre el conjunto de validación
    (no se hace cross validation). La búsqueda en el espacio dehiperparámetros puede realizarse con random
    search o grid search.
    
    Esta función trabaja con clases de modelos de sklearn, xgboost o similares (deben soportar los métodos
    fit y predict).
    
    Se exporta el mejor modelo obtenido con el nombre en './{nombre}_{nombre_dataset}.joblib'.
    
    La función devuelve el mejor modelo obtenido en la búsqueda realizada, sus hiperparámetros su score
    obtenido.
    
    Se tiene opción de paralelizar los entrenamientos en los núcleos de la CPU.
    
    Paámetros:
    ----------
        nombre : str
            nombre con el que se identifica al modelo. Se utilizará en el nombre de la variables
            
        config : dict
            Diccionario con la configuración del modelo. Debe tener los siguientes pares de clave - valor:
                - 'modelo' : Instancia básica del modelo a probar. Debe ser un objeto de sklearn, xgboost o
                alguna librería afín.
                'params' : diccionario. Sus claves son los nombres de los hiperparámetros (dados en la
                documentación de la clase de clasificador / regresor que se utilice) y sus valores
                corresponden a listas con los rangos o valores a ensayar.
                    
        X_train : pandas.DataFrame
            Dataframe del conjunto entrenamiento (sin variable objetivo).
        y_train : pandas.DataFrame | pandas.Serie
            Dataframe con los valores de la variable objetivo del conjunto de entrenamiento.
        X_val : pandas.DataFrame
            Dataframe con las muestras del conjunto de validación (sin variable objetivo).
        y_val : pandas.DataFrame | pandas.Serie
            Dataframe con los valores de la variable objetivo del conjunto de validación.
            
        nombre_dataset : str
            Nombre con el que se identifica al dataset. Se utilizará como sufijo en el nombre final del modelo
            para identificar con qué dataset se entrenó.
            
        metrica : str
            Métrica a utilizar. Consultar opciones válidas en
            https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-string-names. Esta función
            utiliza métricas negativas.
            
        busqueda : str, default='random'
            La forma en que se realizará la búsqueda de hiperparámetros óptimos en el espacio de
            hiperparámetros. Opciones:
                - 'random' : random search
                - 'grid' : grid search.
                
        random_state : int, default=42
            Valor semilla para la elección aleatoria en caso de usar random search.
            Se fija para reproducibilidad.
            
        n_iter : int, default=20
            Cantidad máxima de combinaciones de valores de hiperparámetros a probar en caso de que se use random
            search.
            
        n_jobs : int, default=1
            Cantidad de trabajos a correr en paralelo.
            
    Devuelve:
    ---------
        best_model : sklearn | xgboost model
            Módelo con hiperparámetros óptimos según búsqueda realizada.
        
        best_params : dict
            Diccionario con pares clave - valor dados por los nombres de los hiperparámetros ingresados en
            config['params'] y sus valores óptimos.
            
        best_score : float
            Score obtenido por el modelo óptimo (valor de la métrica sobre X_test).
    """
    start_time = time.time()
    print(f"🚀 Iniciando entrenamiento para: {nombre}")
    
    scorer = get_scorer(metrica)
    
    param_space = config["params"]
    all_param_combinations = list(itertools.product(*param_space.values()))
    param_keys = list(param_space.keys())
    
    total_combinations = 1
    for v in param_space.values():
        total_combinations *= len(v)
    n_iter_ajustado = min(8, total_combinations)

    # Random o grid
    if busqueda == 'grid':
        candidates = all_param_combinations
    elif busqueda == 'random':
        random.seed(random_state)
        candidates = random.sample(all_param_combinations,
                                   min(n_iter, len(all_param_combinations)))
    else:
        raise ValueError("busqueda debe ser 'random' o 'grid'.")
        
    # Función auxiliar para entrenar y evaluar un modelo
    def _fit_and_score(params):
        param_dict = dict(zip(param_keys, params))
        pipeline = Pipeline([('clf', config["modelo"].set_params(**param_dict))])
        pipeline.fit(X_train, y_train)
        score = scorer(pipeline, X_val, y_val)
        return param_dict, score, pipeline
    
    # Paralelizar entrenamiento
    results = Parallel(n_jobs=n_jobs)(
        delayed(_fit_and_score)(params) for params in candidates
    )
        
    # Seleccionar el mejor
    best_params, best_score, best_model = max(results, key=lambda x: x[1])
    best_score = -best_score
            
    print(f"✅ {nombre}: búsqueda finalizada.")
    print(f"🏆 Mejor score en validación: {best_score:.4f}")
    print(f"📌 Mejores hiperparámetros: {best_params}")

    elapsed = time.time() - start_time
    print(f"⏱️ Tiempo entrenando {nombre}: {elapsed:.2f} segundos")

    # Guardar modelo
    dump(best_model, f"./{nombre}_{nombre_dataset}.joblib")

    return best_model, best_params, best_score

## Importo datasets

In [6]:
## Importamos los datasets A

# Etiquetas
y_A_train = pd.read_csv("./A_sets/y_train_inicial.csv")
y_A_test = pd.read_csv("./A_sets/y_test_inicial.csv")
# Datos escaleados
X_A_train_scaled = pd.read_csv("./A_sets/x_A_train_norm.csv")
X_A_test_scaled = pd.read_csv("./A_sets/x_A_test_norm.csv")
# Datos estandarizados
X_A_train_std = pd.read_csv("./A_sets/x_A_train_std.csv")
X_A_test_std = pd.read_csv("./A_sets/x_A_test_std.csv")

In [7]:
## Importamos los datasets B

# Etiquetas
y_B_train = pd.read_csv("./B_sets/y_train_inicial.csv")
y_B_test = pd.read_csv("./B_sets/y_test_inicial.csv")
# Datos escaleados
X_B_train_scaled = pd.read_csv("./B_sets/x_B_train_norm.csv")
X_B_test_scaled = pd.read_csv("./B_sets/x_B_test_norm.csv")
# Datos estandarizados
X_B_train_std = pd.read_csv("./B_sets/x_B_train_std.csv")
X_B_test_std = pd.read_csv("./B_sets/x_B_test_std.csv")

In [8]:
## Defino diccinario para iterar sobre los datasets

datasets_dict = {
    'A_scaled' : {'x_train':X_A_train_scaled, 'y_train':y_A_train, 'x_test':X_A_test_scaled, 'y_test':y_A_test},
    'A_std' : {'x_train':X_A_train_std, 'y_train':y_A_train, 'x_test':X_A_test_std, 'y_test':y_A_test},
    'B_scaled' : {'x_train':X_B_train_scaled, 'y_train':y_B_train, 'x_test':X_B_test_scaled, 'y_test':y_B_test},
    'B_std' : {'x_train':X_B_train_std, 'y_train':y_B_train, 'x_test':X_B_test_std, 'y_test':y_B_test},
}

## Ejemplos de uso

### Regresión polinomial

In [9]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import numpy as np

polynomial_reg_config = {
    "modelo": Pipeline([
        ("poly", PolynomialFeatures()),
        ("linreg", LinearRegression())
    ]),
    "params": {
        "poly__degree": np.arange(1, 3),
        "poly__interaction_only": [True, False],
        "poly__include_bias": [True, False],
        "linreg__positive": [True, False]
    }
}


In [10]:
resultados = {}

for set_conjuntos, set_dict in datasets_dict.items():
    print(f'====================\nDataset {set_conjuntos}\n====================')
    best_model, best_params, best_score = optimizador_hiperparametros(nombre='Regresor_Polinomico',
                                                                      config=polynomial_reg_config,
                                                                      X_train=set_dict['x_train'],
                                                                      y_train=set_dict['y_train'],
                                                                      X_val=set_dict['x_test'],
                                                                      y_val=set_dict['y_test'],
                                                                      nombre_dataset=set_conjuntos,
                                                                      metrica='neg_root_mean_squared_error',
                                                                      busqueda='random',
                                                                      random_state=42,
                                                                      n_iter=20,
                                                                      n_jobs=4)
    resultados[set_conjuntos] = {'mejor_modelo' : best_model,
                                 'mejores_params' : best_params,
                                 'mejor_score' : best_score
                                }
    print("\n")

Dataset A_scaled
🚀 Iniciando entrenamiento para: Regresor_Polinomico
✅ Regresor_Polinomico: búsqueda finalizada.
🏆 Mejor score en validación: 10.4708
📌 Mejores hiperparámetros: {'poly__degree': np.int64(1), 'poly__interaction_only': True, 'poly__include_bias': False, 'linreg__positive': False}
⏱️ Tiempo entrenando Regresor_Polinomico: 5.77 segundos


Dataset A_std
🚀 Iniciando entrenamiento para: Regresor_Polinomico
✅ Regresor_Polinomico: búsqueda finalizada.
🏆 Mejor score en validación: 10.4708
📌 Mejores hiperparámetros: {'poly__degree': np.int64(1), 'poly__interaction_only': True, 'poly__include_bias': True, 'linreg__positive': False}
⏱️ Tiempo entrenando Regresor_Polinomico: 0.67 segundos


Dataset B_scaled
🚀 Iniciando entrenamiento para: Regresor_Polinomico
✅ Regresor_Polinomico: búsqueda finalizada.
🏆 Mejor score en validación: 10.5060
📌 Mejores hiperparámetros: {'poly__degree': np.int64(1), 'poly__interaction_only': True, 'poly__include_bias': False, 'linreg__positive': False}
⏱️ 

### LASSO

In [45]:
from sklearn.linear_model import Lasso
import numpy as np

lasso_config = {
    "modelo": Pipeline([
        ("poly", PolynomialFeatures()),
        ("linreg", Lasso(random_state=42))
    ]),
    "params": {
        "poly__degree": np.arange(1, 6),
        "poly__interaction_only": [True, False],
        "poly__include_bias": [True, False],
        "linreg__positive": [True, False],
        "linreg__alpha": np.logspace(-4, 4, num=9, base=10)
    }
}

In [46]:
resultados = {}

for set_conjuntos, set_dict in datasets_dict.items():
    print(f'====================\nDataset {set_conjuntos}\n====================')
    best_model, best_params, best_score = optimizador_hiperparametros(nombre='LASSO',
                                                                      config=lasso_config,
                                                                      X_train=set_dict['x_train'],
                                                                      y_train=set_dict['y_train'],
                                                                      X_val=set_dict['x_test'],
                                                                      y_val=set_dict['y_test'],
                                                                      nombre_dataset=set_conjuntos,
                                                                      metrica='neg_root_mean_squared_error',
                                                                      busqueda='random',
                                                                      random_state=42,
                                                                      n_iter=20,
                                                                      n_jobs=4)
    resultados[set_conjuntos] = {'mejor_modelo' : best_model,
                                 'mejores_params' : best_params,
                                 'mejor_score' : best_score
                                }
    print("\n")

Dataset A_scaled
🚀 Iniciando entrenamiento para: LASSO


/home/edu/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.727e+04, tolerance: 4.981e+00
  model = cd_fast.enet_coordinate_descent(
/home/edu/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.138e+04, tolerance: 4.981e+00
  model = cd_fast.enet_coordinate_descent(
/home/edu/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.015e+03,

✅ LASSO: búsqueda finalizada.
🏆 Mejor score en validación: 10.4202
📌 Mejores hiperparámetros: {'poly__degree': np.int64(1), 'poly__interaction_only': False, 'poly__include_bias': True, 'linreg__positive': False, 'linreg__alpha': np.float64(0.01)}
⏱️ Tiempo entrenando LASSO: 203.21 segundos


Dataset A_std
🚀 Iniciando entrenamiento para: LASSO


/home/edu/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.015e+01, tolerance: 4.981e+00
  model = cd_fast.enet_coordinate_descent(
/home/edu/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.366e+01, tolerance: 4.981e+00
  model = cd_fast.enet_coordinate_descent(
/home/edu/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.428e+03,

✅ LASSO: búsqueda finalizada.
🏆 Mejor score en validación: 10.3910
📌 Mejores hiperparámetros: {'poly__degree': np.int64(1), 'poly__interaction_only': True, 'poly__include_bias': True, 'linreg__positive': False, 'linreg__alpha': np.float64(0.1)}
⏱️ Tiempo entrenando LASSO: 147.37 segundos


Dataset B_scaled
🚀 Iniciando entrenamiento para: LASSO


/home/edu/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.946e+02, tolerance: 4.981e+00
  model = cd_fast.enet_coordinate_descent(
/home/edu/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.777e+04, tolerance: 4.981e+00
  model = cd_fast.enet_coordinate_descent(
/home/edu/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.050e+04,

✅ LASSO: búsqueda finalizada.
🏆 Mejor score en validación: 10.4581
📌 Mejores hiperparámetros: {'poly__degree': np.int64(1), 'poly__interaction_only': False, 'poly__include_bias': True, 'linreg__positive': False, 'linreg__alpha': np.float64(0.01)}
⏱️ Tiempo entrenando LASSO: 161.15 segundos


Dataset B_std
🚀 Iniciando entrenamiento para: LASSO


/home/edu/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.426e+01, tolerance: 4.981e+00
  model = cd_fast.enet_coordinate_descent(
/home/edu/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.610e+03, tolerance: 4.981e+00
  model = cd_fast.enet_coordinate_descent(
/home/edu/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.407e+03,

✅ LASSO: búsqueda finalizada.
🏆 Mejor score en validación: 10.4344
📌 Mejores hiperparámetros: {'poly__degree': np.int64(1), 'poly__interaction_only': True, 'poly__include_bias': True, 'linreg__positive': False, 'linreg__alpha': np.float64(0.1)}
⏱️ Tiempo entrenando LASSO: 108.08 segundos




---

### XGBoost

#### Uso con XGBoost (espacio reducido, pruebas rápidas)

In [11]:
xgboost_reducido_config = {
    "modelo": XGBRegressor(
        objective="reg:squarederror",
        random_state=42,
        n_jobs=1,
        verbosity=0
    ),
    "params": {
        "n_estimators": [100, 200],
        "max_depth": [3, 5],
        "learning_rate": [0.05, 0.1]
    }
}

In [13]:
resultados = {}

for set_conjuntos, set_dict in datasets_dict.items():
    print(f'====================\nDataset {set_conjuntos}\n====================')
    best_model, best_params, best_score = optimizador_hiperparametros(nombre='XGBoost',
                                                                      config=xgboost_reducido_config,
                                                                      X_train=set_dict['x_train'],
                                                                      y_train=set_dict['y_train'],
                                                                      X_val=set_dict['x_test'],
                                                                      y_val=set_dict['y_test'],
                                                                      nombre_dataset=set_conjuntos,
                                                                      metrica='neg_root_mean_squared_error',
                                                                      busqueda='random',
                                                                      random_state=42,
                                                                      n_iter=20,
                                                                      n_jobs=4)
    resultados[set_conjuntos] = {'mejor_modelo': best_model,
                                 'mejores_params': best_params,
                                 'mejor_score': best_score
                                }
    print("\n")

Dataset A_scaled
🚀 Iniciando entrenamiento para: XGBoost
✅ XGBoost: búsqueda finalizada.
🏆 Mejor score en validación: 6.8887
📌 Mejores hiperparámetros: {'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.1}
⏱️ Tiempo entrenando XGBoost: 2.00 segundos


Dataset A_std
🚀 Iniciando entrenamiento para: XGBoost
✅ XGBoost: búsqueda finalizada.
🏆 Mejor score en validación: 6.8887
📌 Mejores hiperparámetros: {'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.1}
⏱️ Tiempo entrenando XGBoost: 1.68 segundos


Dataset B_scaled
🚀 Iniciando entrenamiento para: XGBoost
✅ XGBoost: búsqueda finalizada.
🏆 Mejor score en validación: 7.1646
📌 Mejores hiperparámetros: {'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.05}
⏱️ Tiempo entrenando XGBoost: 1.58 segundos


Dataset B_std
🚀 Iniciando entrenamiento para: XGBoost
✅ XGBoost: búsqueda finalizada.
🏆 Mejor score en validación: 7.1646
📌 Mejores hiperparámetros: {'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.05}
⏱️ Tiempo entrenan

#### Uso con XGBoost (espacio completo)

In [14]:
from xgboost import XGBRegressor

xgboost_config = {
    "modelo": XGBRegressor(
        objective="reg:squarederror",
        random_state=42,
        n_jobs=1,  # el paralelismo lo controla la función externa
        verbosity=0
    ),
    "params": {
        "n_estimators": [100, 200, 300],
        "max_depth": [3, 5, 7, 9],
        "learning_rate": [0.01, 0.05, 0.1, 0.2],
        "subsample": [0.6, 0.8, 1.0],
        "colsample_bytree": [0.6, 0.8, 1.0],
        "reg_alpha": [0, 0.1, 1, 10],   # L1 regularization
        "reg_lambda": [1, 10, 50]       # L2 regularization
    }
}

In [15]:
resultados = {}

for set_conjuntos, set_dict in datasets_dict.items():
    print(f'====================\nDataset {set_conjuntos}\n====================')
    best_model, best_params, best_score = optimizador_hiperparametros(nombre='XGBoost',
                                                                      config=xgboost_config,
                                                                      X_train=set_dict['x_train'],
                                                                      y_train=set_dict['y_train'],
                                                                      X_val=set_dict['x_test'],
                                                                      y_val=set_dict['y_test'],
                                                                      nombre_dataset=set_conjuntos,
                                                                      metrica='neg_root_mean_squared_error',
                                                                      busqueda='random',
                                                                      random_state=42,
                                                                      n_iter=20,
                                                                      n_jobs=4)
    resultados[set_conjuntos] = {'mejor_modelo': best_model,
                                 'mejores_params': best_params,
                                 'mejor_score': best_score
                                }
    print("\n")


Dataset A_scaled
🚀 Iniciando entrenamiento para: XGBoost
✅ XGBoost: búsqueda finalizada.
🏆 Mejor score en validación: 6.3140
📌 Mejores hiperparámetros: {'n_estimators': 100, 'max_depth': 7, 'learning_rate': 0.1, 'subsample': 0.8, 'colsample_bytree': 1.0, 'reg_alpha': 0.1, 'reg_lambda': 1}
⏱️ Tiempo entrenando XGBoost: 8.61 segundos


Dataset A_std
🚀 Iniciando entrenamiento para: XGBoost
✅ XGBoost: búsqueda finalizada.
🏆 Mejor score en validación: 6.3140
📌 Mejores hiperparámetros: {'n_estimators': 100, 'max_depth': 7, 'learning_rate': 0.1, 'subsample': 0.8, 'colsample_bytree': 1.0, 'reg_alpha': 0.1, 'reg_lambda': 1}
⏱️ Tiempo entrenando XGBoost: 8.37 segundos


Dataset B_scaled
🚀 Iniciando entrenamiento para: XGBoost
✅ XGBoost: búsqueda finalizada.
🏆 Mejor score en validación: 6.3687
📌 Mejores hiperparámetros: {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.1, 'subsample': 0.8, 'colsample_bytree': 1.0, 'reg_alpha': 0.1, 'reg_lambda': 10}
⏱️ Tiempo entrenando XGBoost: 8.48 segun